# Plan

Goals:
- Win a medal (macro-F1). Baseline quickly, then iterate.

High-level approach:
1) Environment checks and setup
   - Verify GPU availability; install correct Torch stack (cu121).
   - Set up core deps: timm, albumentations, torchvision, pandas, scikit-learn.

2) Data understanding
   - Load train.csv/test.csv; inspect schema, counts, unique category_id, class imbalance.
   - Verify image paths and existence after extracting zips.

3) Validation protocol
   - Stratified k-fold by category_id.
   - Consider group-aware split if location or sequence metadata exists in CSV to reduce leakage; otherwise stick to stratified KFold with multiple seeds.

4) Baseline model (fast)
   - Image-only baseline with pretrained CNN (e.g., tf_efficientnet_b0/b3, resnet50) in PyTorch + timm.
   - 224px, lightweight augmentations, label-smoothing cross-entropy, cosine schedule, early stopping.
   - 5-fold, save OOF and test logits.

5) Improve
   - Increase res (256/320), stronger augs (RandomResizedCrop, ColorJitter, RandomErasing), mixup/cutmix.
   - Try better backbones (convnext_tiny, eva02_tiny if available), EMA, balanced sampler/focal loss for imbalance.
   - Calibrate thresholds per-class from OOF if needed for macro-F1.

6) Ensembling
   - Blend diverse backbones/seeds/resolutions via OOF-driven weights.

7) Error analysis
   - Mine OOF by class; apply class-weighting, focal loss, TTA for weak classes.

Risk controls / efficiency:
- Always log progress with elapsed time.
- Subsample smoke runs (1 fold, 2-3 epochs) to validate pipeline before full training.
- Cache datasets/transforms; avoid re-reading images unnecessarily.

Requests for expert review:
- Confirm best medal-winning backbones and input sizes for this dataset.
- Recommended validation (any grouping like location_id/sequence_id in CSV?).
- Loss/augmentation settings that historically worked (focal vs LSCE, mixup/cutmix ratios).
- Optimal TTA and ensembling strategy.

---

Next: run environment and data peek cell.

In [2]:
# Environment and quick data peek
import os, sys, subprocess, platform
import pandas as pd

print('Python', sys.version)
print('Platform', platform.platform())

def run(cmd):
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout

print('nvidia-smi:')
print(run(['bash','-lc','nvidia-smi || true']))

train_path = 'train.csv'
test_path = 'test.csv'
assert os.path.exists(train_path) and os.path.exists(test_path), 'CSV files missing'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
print('Train shape:', train.shape)
print('Test shape:', test.shape)
print('Train columns:', train.columns.tolist())
print('Test columns:', test.columns.tolist())
print('\nTrain head:')
print(train.head())
print('\nTest head:')
print(test.head())

target_col = 'category_id'
if target_col in train.columns:
    n_classes = train[target_col].nunique()
    print('Unique classes:', n_classes)
    print('Class distribution (top 20):')
    print(train[target_col].value_counts().head(20))
else:
    print('Target column not found!')

img_cols = [c for c in train.columns if any(k in c.lower() for k in ['image','file','path'])]
print('Possible image columns:', img_cols)

Python 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
Platform Linux-6.8.0-1031-azure-x86_64-with-glibc2.35
nvidia-smi:
Sat Sep 27 02:53:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|             

Train shape: (179422, 11)
Test shape: (16877, 10)
Train columns: ['category_id', 'date_captured', 'file_name', 'frame_num', 'id', 'location', 'rights_holder', 'seq_id', 'seq_num_frames', 'width', 'height']
Test columns: ['date_captured', 'file_name', 'frame_num', 'id', 'location', 'rights_holder', 'seq_id', 'seq_num_frames', 'width', 'height']

Train head:
   category_id        date_captured                                 file_name  \
0           19  2012-03-17 03:48:44  588a679f-23d2-11e8-a6a3-ec086b02610b.jpg   
1            0  2014-05-11 11:56:46  59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg   
2            0  2013-10-06 02:00:00  5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg   
3            3  2011-06-28 15:29:42  593d68d7-23d2-11e8-a6a3-ec086b02610b.jpg   
4            0  2014-07-31 14:00:00  58782b45-23d2-11e8-a6a3-ec086b02610b.jpg   

   frame_num                                    id  location  rights_holder  \
0          2  588a679f-23d2-11e8-a6a3-ec086b02610b       115   Justin Brown

In [3]:
# Setup Torch cu121 stack, deps, and unzip images
import os, sys, subprocess, shutil, time, glob, random, pathlib, json
from pathlib import Path

def run(cmd):
    print('>', ' '.join(cmd), flush=True)
    return subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, check=False).stdout

# 0) Hard reset prior torch stacks
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
):
    if os.path.exists(d):
        print("Removing", d); shutil.rmtree(d, ignore_errors=True)

# 1) Install exact cu121 torch stack
print('Installing torch/cu121 stack...', flush=True)
out = run([sys.executable, "-m", "pip", "install",
           "--index-url", "https://download.pytorch.org/whl/cu121",
           "--extra-index-url", "https://pypi.org/simple",
           "torch==2.4.1", "torchvision==0.19.1", "torchaudio==2.4.1"])
print(out[-1000:])

Path("constraints.txt").write_text("torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n")

# 2) Non-torch deps
print('Installing deps...', flush=True)
out = run([sys.executable, "-m", "pip", "install", "-c", "constraints.txt",
           "timm==1.0.9", "albumentations==1.4.14", "opencv-python-headless==4.10.0.84",
           "scikit-learn", "pandas", "numpy", "Pillow", "tqdm",
           "--upgrade-strategy", "only-if-needed"])
print(out[-1000:])

import torch
print("torch:", torch.__version__, "CUDA:", getattr(torch.version, "cuda", None), "CUDA available:", torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f"Wrong CUDA build: {torch.version.cuda}"

# 3) Unzip images if needed
def unzip_if_needed(zip_path, out_dir):
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    # Heuristic: if directory has > 10 jpgs, assume extracted
    existing = list(out.glob('*.jpg'))
    if len(existing) > 10:
        print(f"{out_dir} already extracted with {len(existing)} jpgs.")
        return
    assert Path(zip_path).exists(), f"Missing {zip_path}"
    print(f"Extracting {zip_path} -> {out_dir} ...", flush=True)
    t0 = time.time()
    cmd = ["bash","-lc", f"unzip -q -o {zip_path} -d {out_dir}"]
    print(run(cmd)[-1000:])
    print(f"Done in {time.time()-t0:.1f}s. Files: {len(list(Path(out_dir).glob('*.jpg')))}")

unzip_if_needed('train_images.zip', 'train_images')
unzip_if_needed('test_images.zip', 'test_images')

# 4) Quick check a few random images exist
train_imgs = sorted(glob.glob('train_images/*.jpg'))
test_imgs = sorted(glob.glob('test_images/*.jpg'))
print('Train images:', len(train_imgs), 'Test images:', len(test_imgs))
print('Sample train:', train_imgs[:3])
print('Sample test:', test_imgs[:3])

print('Setup complete.')

Installing torch/cu121 stack...


> /usr/bin/python3.11 -m pip install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


pmath, typing-extensions, sympy, pillow, nvidia-nvtx-cu12, nvidia-nvjitlink-cu12, nvidia-nccl-cu12, nvidia-curand-cu12, nvidia-cufft-cu12, nvidia-cuda-runtime-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-cupti-cu12, nvidia-cublas-cu12, numpy, networkx, MarkupSafe, fsspec, filelock, triton, nvidia-cusparse-cu12, nvidia-cudnn-cu12, jinja2, nvidia-cusolver-cu12, torch, torchvision, torchaudio

Installing deps...


> /usr/bin/python3.11 -m pip install -c constraints.txt timm==1.0.9 albumentations==1.4.14 opencv-python-headless==4.10.0.84 scikit-learn pandas numpy Pillow tqdm --upgrade-strategy only-if-needed


 directory /app/.pip-target/sympy already exists. Specify --upgrade to force replacement.



torch: 2.4.1+cu121 CUDA: 12.1 CUDA available: True
Extracting train_images.zip -> train_images ...


> bash -lc unzip -q -o train_images.zip -d train_images


bash: line 1: unzip: command not found

Done in 0.0s. Files: 0
Extracting test_images.zip -> test_images ...


> bash -lc unzip -q -o test_images.zip -d test_images


bash: line 1: unzip: command not found

Done in 0.0s. Files: 0
Train images: 0 Test images: 0
Sample train: []
Sample test: []
Setup complete.


In [4]:
# Fallback extraction with Python's zipfile (no system unzip)
import zipfile, os, glob, time
from pathlib import Path

def py_unzip(zip_path, out_dir):
    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    existing = list(out.glob('*.jpg'))
    if len(existing) > 10:
        print(f"{out_dir} already has {len(existing)} jpgs; skipping.")
        return
    assert Path(zip_path).exists(), f"Missing {zip_path}"
    t0 = time.time()
    with zipfile.ZipFile(zip_path) as zf:
        members = zf.namelist()
        print(f"Extracting {len(members)} files from {zip_path} ...", flush=True)
        zf.extractall(out_dir)
    njpg = len(list(Path(out_dir).glob('*.jpg')))
    print(f"Done in {time.time()-t0:.1f}s. JPGs: {njpg}")

py_unzip('train_images.zip', 'train_images')
py_unzip('test_images.zip', 'test_images')
print('Post-extract counts:', len(glob.glob('train_images/*.jpg')), len(glob.glob('test_images/*.jpg')))

Extracting 179224 files from train_images.zip ...


Done in 98.5s. JPGs: 179224
Extracting 16862 files from test_images.zip ...


Done in 7.9s. JPGs: 16862


Post-extract counts: 179224 16862


In [8]:
# CV split, label mapping, and dataset/transforms setup
import pandas as pd, numpy as np, os
from sklearn.model_selection import StratifiedGroupKFold
from collections import Counter
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import torch
from torch.utils.data import Dataset

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Label mapping: model indices [0..C-1] <-> original category_id
cats = np.sort(train['category_id'].unique())
cat2idx = {c:i for i,c in enumerate(cats)}
idx2cat = {i:c for c,i in cat2idx.items()}
train['label'] = train['category_id'].map(cat2idx).astype(int)
num_classes = len(cats)
print('Num classes:', num_classes)

# Save mapping for later use
pd.Series(idx2cat).to_csv('idx2cat.csv', header=['category_id'])
pd.Series(cat2idx).to_csv('cat2idx.csv', header=['idx'])

# StratifiedGroupKFold by location (primary per expert advice)
n_splits = 5
sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
folds = np.full(len(train), -1, dtype=int)
y = train['label'].values
groups = train['location'].values
for f,(tr,va) in enumerate(sgkf.split(train, y, groups)):
    folds[va] = f
assert (folds>=0).all()
train['fold'] = folds
train.to_csv('train_folds.csv', index=False)
print('Fold sizes:', Counter(folds))

# Basic transforms @320
IMG_SIZE = 320
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_tfms = A.Compose([
    A.RandomResizedCrop(IMG_SIZE, IMG_SIZE, scale=(0.6,1.0), interpolation=Image.BICUBIC),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(0.2,0.2,0.1,0.0,p=0.5),
    A.Normalize(mean=mean, std=std),
    A.CoarseDropout(max_holes=1, max_height=int(IMG_SIZE*0.2), max_width=int(IMG_SIZE*0.2), p=0.3),
    ToTensorV2(),
])
val_tfms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE, interpolation=Image.BICUBIC),
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])

class ImgDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
        self.has_label = 'label' in df.columns
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['file_name']
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception:
            # Fallback: black image if missing/corrupt
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        if self.has_label:
            return img, int(row['label'])
        else:
            return img, row['id']

print('CV and dataset setup complete. Ready for smoke training next.')

Num classes: 14


Fold sizes: Counter({4: 77411, 3: 46425, 1: 35471, 0: 13121, 2: 6994})
CV and dataset setup complete. Ready for smoke training next.


In [7]:
# Fix albumentations/albucore mismatch by forcing a clean reinstall of albumentations==1.3.1
import sys, subprocess
def run(cmd):
    print('>', ' '.join(cmd), flush=True)
    out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True).stdout
    print(out[-2000:])
    return out

# Uninstall conflicting packages
run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'albumentations', 'albucore'])
# Install pinned albumentations without albucore dependency
run([sys.executable, '-m', 'pip', 'install', '-c', 'constraints.txt', 'albumentations==1.3.1', '--no-cache-dir', '--upgrade'])

import albumentations as A
from albumentations.pytorch import ToTensorV2
print('Albumentations version:', A.__version__)

> /usr/bin/python3.11 -m pip uninstall -y albumentations albucore


Found existing installation: albumentations 1.4.14
Uninstalling albumentations-1.4.14:
  Successfully uninstalled albumentations-1.4.14
Found existing installation: albucore 0.0.33
Uninstalling albucore-0.0.33:
  Successfully uninstalled albucore-0.0.33

> /usr/bin/python3.11 -m pip install -c constraints.txt albumentations==1.3.1 --no-cache-dir --upgrade


━━━━━━━━ 44.6/44.6 KB 417.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 209.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 192.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 302.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 457.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 KB 470.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 KB 553.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 KB 543.5 MB/s eta 0:00:00



Albumentations version: 1.3.1


In [9]:
# Smoke training: convnext_tiny @320, 1 fold, 1 epoch, AMP + sampler
import time, math, numpy as np, pandas as pd, torch, timm
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_folds = pd.read_csv('train_folds.csv')
num_classes = train_folds['label'].nunique()
print('Classes:', num_classes, 'Device:', device)

# Fold selection (smoke: use fold==0 as val)
VAL_FOLD = 0
df_tr = train_folds[train_folds.fold != VAL_FOLD].copy()
df_va = train_folds[train_folds.fold == VAL_FOLD].copy()
print('Train/Val sizes:', len(df_tr), len(df_va))

# Datasets
train_ds = ImgDataset(df_tr, 'train_images', transform=train_tfms)
val_ds = ImgDataset(df_va, 'train_images', transform=val_tfms)

# Sampler: class weights ~ 1/sqrt(count); downweight class 0
counts = df_tr['label'].value_counts().to_dict()
cls_w = {c: 1.0 / math.sqrt(counts.get(c, 1)) for c in range(num_classes)}
empty_idx = int((pd.Series(idx2cat).sort_index().to_dict()).get(0, 0))  # original cat 0 -> model idx
if empty_idx in cls_w:
    cls_w[empty_idx] *= 0.3
weights = df_tr['label'].map(cls_w).values
sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

# Loaders
BATCH_SIZE = 32
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# Model
model = timm.create_model('convnext_tiny', pretrained=True, num_classes=num_classes)
model.to(device)

# Loss (label smoothing CE) and optimizer/scheduler
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=0.03)
EPOCHS = 1  # smoke
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

# Simple EMA
ema_decay = 0.999
ema_params = [p.detach().clone() for p in model.parameters() if p.requires_grad]
def ema_update():
    with torch.no_grad():
        i = 0
        for p in model.parameters():
            if not p.requires_grad: continue
            ema_params[i].mul_(ema_decay).add_(p.detach(), alpha=1-ema_decay)
            i += 1
def swap_to_ema(store):
    i = 0
    for p in model.parameters():
        if not p.requires_grad: continue
        store.append(p.detach().clone())
        p.data.copy_(ema_params[i])
        i += 1

def evaluate(loader):
    model.eval()
    all_preds, all_tgts = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(xb)
            preds = logits.argmax(1).detach().cpu().numpy().tolist()
            all_preds.extend(preds)
            all_tgts.extend(yb.detach().cpu().numpy().tolist())
    return f1_score(all_tgts, all_preds, average='macro')

# Train loop
t0 = time.time()
for epoch in range(EPOCHS):
    model.train()
    running_loss, n, t_ep = 0.0, 0, time.time()
    for it, (xb, yb) in enumerate(train_loader):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        ema_update()
        running_loss += loss.item() * xb.size(0)
        n += xb.size(0)
        if (it+1) % 100 == 0:
            print(f"Epoch {epoch+1} Iter {it+1} | loss {running_loss/max(1,n):.4f} | elapsed {time.time()-t_ep:.1f}s", flush=True)
    scheduler.step()
    # Eval with EMA weights
    backup = []
    swap_to_ema(backup)
    val_f1 = evaluate(val_loader)
    # restore weights
    i = 0
    for p in model.parameters():
        if not p.requires_grad: continue
        p.data.copy_(backup[i]); i += 1
    print(f"Epoch {epoch+1}/{EPOCHS} | train_loss {running_loss/max(1,n):.4f} | val_macroF1 {val_f1:.4f} | epoch_time {time.time()-t_ep:.1f}s", flush=True)

print(f"Total time: {time.time()-t0:.1f}s")
print('Smoke training complete.')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classes: 14 Device: cuda
Train/Val sizes: 166301 13121


/tmp/ipykernel_94/3344867879.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/tmp/ipykernel_94/3344867879.py:90: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 1 Iter 100 | loss 2.5875 | elapsed 13.5s


...

Epoch 1 Iter 5000 | loss 1.7312 | elapsed 647.7s


Epoch 1 Iter 5100 | loss 1.7229 | elapsed 660.7s


/tmp/ipykernel_94/3344867879.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 1/1 | train_loss 1.7153 | val_macroF1 0.1710 | epoch_time 697.2s


Total time: 697.2s
Smoke training complete.


In [10]:
# Inference: TTA(hflip), sequence averaging, and submission
import pandas as pd, numpy as np, torch
from torch.utils.data import DataLoader
from pathlib import Path

test_df = pd.read_csv('test.csv').copy()

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['file_name']
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception:
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = np.array(img)
        x = self.transform(image=img)['image']
        return x, row['id'], row['seq_id']

model.eval()
test_ds = TestDataset(test_df, 'test_images', transform=val_tfms)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

all_logits = []
all_ids = []
all_seqs = []
with torch.no_grad():
    for xb, ids, seqs in test_loader:
        xb = xb.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            # TTA: horizontal flip
            xb_flip = torch.flip(xb, dims=[-1])
            logits_flip = model(xb_flip)
            logits = 0.5 * (logits + logits_flip)
        all_logits.append(logits.detach().cpu())
        all_ids.extend(ids)
        all_seqs.extend(seqs)

all_logits = torch.cat(all_logits, dim=0).numpy()
preds_df = pd.DataFrame({
    'id': all_ids,
    'seq_id': all_seqs
})
for j in range(all_logits.shape[1]):
    preds_df[f'logit_{j}'] = all_logits[:, j]

# Sequence-level averaging of logits
logit_cols = [c for c in preds_df.columns if c.startswith('logit_')]
seq_avg = preds_df.groupby('seq_id')[logit_cols].mean().reset_index()
preds_df = preds_df.drop(columns=logit_cols).merge(seq_avg, on='seq_id', how='left')

# Argmax to model label, then map back to original category_id
logits_mat = preds_df[logit_cols].values
model_preds = logits_mat.argmax(axis=1)
idx2cat_map = pd.read_csv('idx2cat.csv', index_col=0)['category_id'].to_dict()
cat_preds = [idx2cat_map[int(i)] for i in model_preds]

submission = pd.DataFrame({'id': preds_df['id'], 'category_id': cat_preds})
submission.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape:', submission.shape)
print(submission.head())

/tmp/ipykernel_94/3922678961.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Saved submission.csv with shape: (16877, 2)
                                     id  category_id
0  5998cfa4-23d2-11e8-a6a3-ec086b02610b           13
1  599fbd89-23d2-11e8-a6a3-ec086b02610b           11
2  59fae563-23d2-11e8-a6a3-ec086b02610b           16
3  5a24a741-23d2-11e8-a6a3-ec086b02610b           18
4  59eab924-23d2-11e8-a6a3-ec086b02610b           16


In [11]:
# Train fold with Balanced Softmax, EMA, seq-avg val, checkpoint, and OOF/test logits saving
import os, math, time, json
import numpy as np, pandas as pd, torch, timm
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from sklearn.metrics import f1_score
from pathlib import Path
from collections import Counter
from albumentations.pytorch import ToTensorV2
import albumentations as A
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load folds and mappings
df_all = pd.read_csv('train_folds.csv')
cat2idx_map = pd.read_csv('cat2idx.csv', index_col=0)['idx'].to_dict()
idx2cat_map = pd.read_csv('idx2cat.csv', index_col=0)['category_id'].to_dict()
num_classes = df_all['label'].nunique()
empty_idx = cat2idx_map.get(0, None)  # correct empty mapping
print('num_classes:', num_classes, 'empty_idx:', empty_idx)

# Compute dataset priors for Balanced Softmax
class_counts = df_all['label'].value_counts().reindex(range(num_classes)).fillna(0).astype(int).values
priors = class_counts / class_counts.sum()
log_priors = torch.log(torch.tensor(priors + 1e-12, dtype=torch.float32, device=device))

class ValDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.img_dir / row['file_name']
        try:
            img = Image.open(img_path).convert('RGB')
        except Exception:
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = np.array(img)
        x = self.transform(image=img)['image'] if self.transform else img
        return x, int(row['label']), row['seq_id'], idx  # return local idx for OOF alignment

def make_sampler(df_tr):
    counts = df_tr['label'].value_counts().to_dict()
    w = {c: 1.0 / math.sqrt(counts.get(c, 1)) for c in range(num_classes)}
    if empty_idx is not None and empty_idx in w:
        w[empty_idx] *= 0.3  # downweight empty
    weights = df_tr['label'].map(w).astype('float32').values
    return WeightedRandomSampler(weights=weights, num_samples=len(weights), replacement=True)

def balanced_softmax_ce(logits, targets):
    # logits: (B, C), targets: (B,)
    return nn.functional.cross_entropy(logits + log_priors, targets)

def seq_avg_macro_f1(pred_logits, true_labels, seq_ids):
    # pred_logits: (N, C) numpy; true_labels: (N,), seq_ids: (N,) strings
    dfp = pd.DataFrame({'seq_id': seq_ids})
    logit_cols = [f'l{i}' for i in range(pred_logits.shape[1])]
    for i in range(pred_logits.shape[1]): dfp[logit_cols[i]] = pred_logits[:, i]
    dfp['label'] = true_labels
    seq_mean = dfp.groupby('seq_id')[logit_cols + ['label']].mean().reset_index()
    y_true = seq_mean['label'].astype(int).values
    y_pred = seq_mean[logit_cols].values.argmax(1)
    return f1_score(y_true, y_pred, average='macro')

def train_one_fold(fold=0, epochs=3, batch_size=32, lr=1e-3, wd=0.03, ema_decay=0.9996, save_dir='artifacts'):
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    df_tr = df_all[df_all.fold != fold].copy()
    df_va = df_all[df_all.fold == fold].copy()
    print(f'Fold {fold}: train {len(df_tr)} val {len(df_va)}')

    train_ds = ImgDataset(df_tr, 'train_images', transform=train_tfms)
    val_ds = ValDataset(df_va, 'train_images', transform=val_tfms)
    sampler = make_sampler(df_tr)
    train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    model = timm.create_model('convnext_tiny', pretrained=True, num_classes=num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    # cosine with warmup (1 epoch warmup simple linear)
    total_steps = epochs * max(1, len(train_loader))
    warmup_steps = max(1, len(train_loader))  # ~1 epoch
    def lr_lambda(step):
        if step < warmup_steps:
            return float(step + 1) / warmup_steps
        t = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
        return 0.5 * (1 + math.cos(math.pi * t))
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    # EMA
    ema_params = [p.detach().clone() for p in model.parameters() if p.requires_grad]
    def ema_update():
        with torch.no_grad():
            i = 0
            for p in model.parameters():
                if not p.requires_grad: continue
                ema_params[i].mul_(ema_decay).add_(p.detach(), alpha=1-ema_decay)
                i += 1
    def swap_to_ema(store):
        i = 0
        for p in model.parameters():
            if not p.requires_grad: continue
            store.append(p.detach().clone())
            p.data.copy_(ema_params[i])
            i += 1

    best_f1, best_path = -1.0, str(Path(save_dir) / f'convnext_tiny_fold{fold}.pth')
    global_step = 0
    t0 = time.time()
    for ep in range(epochs):
        model.train()
        ep_loss, seen, t_ep = 0.0, 0, time.time()
        for it, (xb, yb) in enumerate(train_loader):
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model(xb)
                loss = balanced_softmax_ce(logits, yb)
            scaler.scale(loss).backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            ema_update()
            bs = xb.size(0); ep_loss += loss.item() * bs; seen += bs; global_step += 1
            if (it+1) % 200 == 0:
                print(f'E{ep+1} It{it+1} | lr {scheduler.get_last_lr()[0]:.6f} | loss {ep_loss/max(1,seen):.4f} | {time.time()-t_ep:.1f}s', flush=True)

        # Eval with EMA
        backup = []; swap_to_ema(backup); model.eval()
        va_logits_list, va_labels, va_seq = [], [], []
        with torch.no_grad():
            for xb, yb, seqs, idxs in val_loader:
                xb = xb.to(device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    lg = model(xb).detach().cpu().float()
                va_logits_list.append(lg)
                va_labels.extend(yb.numpy().tolist())
                va_seq.extend(seqs)
        # restore params
        i = 0
        for p in model.parameters():
            if not p.requires_grad: continue
            p.data.copy_(backup[i]); i += 1

        va_logits = torch.cat(va_logits_list, dim=0).numpy()
        va_f1 = seq_avg_macro_f1(va_logits, np.array(va_labels), np.array(va_seq))
        print(f'Epoch {ep+1}/{epochs} | train_loss {ep_loss/max(1,seen):.4f} | val_macroF1_seqavg {va_f1:.4f} | ep_time {time.time()-t_ep:.1f}s')

        # Save best
        if va_f1 > best_f1:
            best_f1 = va_f1
            torch.save({'state_dict': model.state_dict(), 'ema_params': [p.cpu() for p in ema_params], 'val_f1': best_f1}, best_path)
            # Save OOF logits for this fold
            np.save(Path(save_dir)/f'oof_logits_fold{fold}.npy', va_logits)
            np.save(Path(save_dir)/f'oof_labels_fold{fold}.npy', np.array(va_labels))
            pd.Series(va_seq).to_csv(Path(save_dir)/f'oof_seq_fold{fold}.csv', index=False, header=['seq_id'])
    print(f'Fold {fold} done. Best val_macroF1_seqavg={best_f1:.4f}. ckpt={best_path} | total {time.time()-t0:.1f}s')
    return best_f1, best_path

# Run a first improved fold-0 training (3 epochs) to validate BSCE setup quickly
best_f1, best_path = train_one_fold(fold=0, epochs=3, batch_size=32, lr=1e-3, wd=0.03, ema_decay=0.9996, save_dir='artifacts')
print('Finished fold0 quick run. Best F1:', best_f1)

num_classes: 14 empty_idx: 0
Fold 0: train 166301 val 13121


/tmp/ipykernel_94/1785543454.py:91: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/app/.pip-target/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


E1 It200 | lr 0.000039 | loss 1.4582 | 25.8s


E1 It400 | lr 0.000077 | loss 1.1201 | 51.3s


...

E3 It4400 | lr 0.000014 | loss 0.1692 | 570.9s


E3 It4600 | lr 0.000008 | loss 0.1679 | 596.7s


E3 It4800 | lr 0.000004 | loss 0.1663 | 622.7s


E3 It5000 | lr 0.000001 | loss 0.1647 | 648.6s


Epoch 3/3 | train_loss 0.1634 | val_macroF1_seqavg 0.4403 | ep_time 693.8s
Fold 0 done. Best val_macroF1_seqavg=0.4424. ckpt=artifacts/convnext_tiny_fold0.pth | total 2083.2s
Finished fold0 quick run. Best F1: 0.4424157099420091


In [12]:
# EMA-based inference from saved checkpoint: hflip TTA + seq-avg + save logits and submission
import torch, pandas as pd, numpy as np
from torch.utils.data import DataLoader
from pathlib import Path

def load_model_with_ema(ckpt_path, num_classes):
    model = timm.create_model('convnext_tiny', pretrained=False, num_classes=num_classes).to(device)
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt['state_dict'])
    # swap to EMA params
    ema_list = ckpt.get('ema_params', None)
    if ema_list is not None:
        i = 0
        with torch.no_grad():
            for p in model.parameters():
                if not p.requires_grad: continue
                p.data.copy_(ema_list[i].to(device)); i += 1
    model.eval()
    return model

def infer_test_and_save(ckpt_path, out_dir='artifacts', alpha_prior: float = 0.0):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    test_df = pd.read_csv('test.csv')
    model = load_model_with_ema(ckpt_path, num_classes)
    test_ds = TestDataset(test_df, 'test_images', transform=val_tfms)
    loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
    all_logits, ids, seqs = [], [], []
    with torch.no_grad():
        for xb, idb, seqb in loader:
            xb = xb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                lg = model(xb)
                lg_flip = model(torch.flip(xb, dims=[-1]))
                lg = 0.5 * (lg + lg_flip)
                if alpha_prior != 0.0:
                    # apply optional logit adjustment by priors
                    lg = lg + alpha_prior * log_priors
            all_logits.append(lg.detach().cpu())
            ids.extend(idb); seqs.extend(seqb)
    logits = torch.cat(all_logits, dim=0).numpy()
    np.save(Path(out_dir)/'test_logits_fold0.npy', logits)
    pred_df = pd.DataFrame({'id': ids, 'seq_id': seqs})
    for j in range(logits.shape[1]):
        pred_df[f'logit_{j}'] = logits[:, j]
    # sequence-average
    logit_cols = [c for c in pred_df.columns if c.startswith('logit_')]
    seq_avg = pred_df.groupby('seq_id')[logit_cols].mean().reset_index()
    pred_df = pred_df.drop(columns=logit_cols).merge(seq_avg, on='seq_id', how='left')
    mat = pred_df[logit_cols].values
    model_idx = mat.argmax(1)
    idx2cat_map = pd.read_csv('idx2cat.csv', index_col=0)['category_id'].to_dict()
    cat_preds = [idx2cat_map[int(i)] for i in model_idx]
    sub = pd.DataFrame({'id': pred_df['id'], 'category_id': cat_preds})
    sub.to_csv('submission.csv', index=False)
    print('Saved submission.csv', sub.shape, 'and test logits to', out_dir)

# After training completes, run:
# infer_test_and_save('artifacts/convnext_tiny_fold0.pth', out_dir='artifacts', alpha_prior=0.0)

In [13]:
# Run EMA-based inference for fold0 and write submission.csv
infer_test_and_save('artifacts/convnext_tiny_fold0.pth', out_dir='artifacts', alpha_prior=0.0)

/tmp/ipykernel_94/1306181635.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Saved submission.csv (16877, 2) and test logits to artifacts


In [14]:
# Train folds 1-2 (8 epochs) then ensemble fold0-2 EMA checkpoints for test submission
from pathlib import Path
import numpy as np, pandas as pd, torch
from torch.utils.data import DataLoader

# 1) Train additional folds (1 and 2) with the improved recipe
folds_to_train = [1, 2]
ckpt_paths = [Path('artifacts')/f'convnext_tiny_fold{f}.pth' for f in folds_to_train]
for f, ck in zip(folds_to_train, ckpt_paths):
    if ck.exists():
        print(f'Skipping fold {f}, checkpoint exists:', ck)
        continue
    print(f'Training fold {f} for 8 epochs...')
    train_one_fold(fold=f, epochs=8, batch_size=32, lr=1e-3, wd=0.03, ema_decay=0.9996, save_dir='artifacts')

# 2) Ensemble EMA checkpoints from folds [0,1,2] if available
all_ckpts = [Path('artifacts')/f'convnext_tiny_fold{i}.pth' for i in [0,1,2]]
all_ckpts = [str(p) for p in all_ckpts if p.exists()]
print('Ensembling checkpoints:', all_ckpts)
assert len(all_ckpts) >= 1, 'No checkpoints found to ensemble'

test_df = pd.read_csv('test.csv')
test_ds = TestDataset(test_df, 'test_images', transform=val_tfms)
loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

logits_list = []
with torch.no_grad():
    for ck in all_ckpts:
        print('Infer with', ck)
        model_e = load_model_with_ema(ck, num_classes)
        part_logits = []
        for xb, idb, seqb in loader:
            xb = xb.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                lg = model_e(xb)
                lgf = model_e(torch.flip(xb, dims=[-1]))
                lg = 0.5*(lg+lgf)
            part_logits.append(lg.detach().cpu())
        logits_list.append(torch.cat(part_logits, dim=0).numpy())

blended = np.mean(np.stack(logits_list, axis=0), axis=0)
np.save(Path('artifacts')/'test_logits_fold0_2_blend.npy', blended)

# Sequence average and write submission
pred_df = pd.DataFrame({'id': test_df['id'].values, 'seq_id': test_df['seq_id'].values})
for j in range(blended.shape[1]):
    pred_df[f'logit_{j}'] = blended[:, j]
logit_cols = [c for c in pred_df.columns if c.startswith('logit_')]
seq_avg = pred_df.groupby('seq_id')[logit_cols].mean().reset_index()
pred_df = pred_df.drop(columns=logit_cols).merge(seq_avg, on='seq_id', how='left')
mat = pred_df[logit_cols].values
model_idx = mat.argmax(1)
idx2cat_map = pd.read_csv('idx2cat.csv', index_col=0)['category_id'].to_dict()
cat_preds = [idx2cat_map[int(i)] for i in model_idx]
sub = pd.DataFrame({'id': pred_df['id'], 'category_id': cat_preds})
sub.to_csv('submission.csv', index=False)
print('Wrote blended submission.csv', sub.shape, 'using', len(all_ckpts), 'checkpoints')

Training fold 1 for 8 epochs...
Fold 1: train 143951 val 35471


/tmp/ipykernel_94/1785543454.py:91: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/app/.pip-target/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


E1 It200 | lr 0.000045 | loss 1.5544 | 26.0s


E1 It400 | lr 0.000089 | loss 1.1627 | 51.7s


...

Fold 2 done. Best val_macroF1_seqavg=0.5316. ckpt=artifacts/convnext_tiny_fold2.pth | total 5682.8s
Ensembling checkpoints: ['artifacts/convnext_tiny_fold0.pth', 'artifacts/convnext_tiny_fold1.pth', 'artifacts/convnext_tiny_fold2.pth']
Infer with artifacts/convnext_tiny_fold0.pth


/tmp/ipykernel_94/1306181635.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Infer with artifacts/convnext_tiny_fold1.pth


Infer with artifacts/convnext_tiny_fold2.pth


Wrote blended submission.csv (16877, 2) using 3 checkpoints


In [15]:
# Fix submission.csv to match required headers: Id, Category
import pandas as pd
sub = pd.read_csv('submission.csv')
rename_map = {}
if 'id' in sub.columns: rename_map['id'] = 'Id'
if 'category_id' in sub.columns: rename_map['category_id'] = 'Category'
if rename_map:
    sub = sub.rename(columns=rename_map)
# Ensure correct column order and dtypes
expected_cols = ['Id', 'Category']
missing = [c for c in expected_cols if c not in sub.columns]
assert not missing, f'Missing required columns: {missing}'
sub = sub[expected_cols]
sub['Category'] = sub['Category'].astype(int)
sub.to_csv('submission.csv', index=False)
print('Fixed submission.csv. Columns:', sub.columns.tolist(), 'Shape:', sub.shape)
print(sub.head())

Fixed submission.csv. Columns: ['Id', 'Category'] Shape: (16877, 2)
                                     Id  Category
0  5998cfa4-23d2-11e8-a6a3-ec086b02610b        19
1  599fbd89-23d2-11e8-a6a3-ec086b02610b         0
2  59fae563-23d2-11e8-a6a3-ec086b02610b         4
3  5a24a741-23d2-11e8-a6a3-ec086b02610b         0
4  59eab924-23d2-11e8-a6a3-ec086b02610b         0
